# GOOD MODEL

In [2]:
import random
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# ----------------- CONFIG -----------------
DATA_PATH = "./data/translations.csv"
TARGET = "checked"
ONNX_OUTPUT = "model_1.onnx"
# ------------------------------------------

# ---- Sensitive features (we keep them but reduce their influence) ----
sensitive_features = [
    "person_gender_woman",
    "person_age_at_investigation",
    "relationship_child_age_difference_parent_first_child",
    "relationship_child_current_number",
    "relationship_child_primary_school_child",
    "relationship_child_has_children",
    "relationship_child_young_adult",
    "relationship_child_teen",
    "relationship_child_adult",
    "relationship_other_current_form_cost_sharer",
    "relationship_other_current_form_parents_caregivers",
    "relationship_other_current_form_other",
    "relationship_other_current_form_authorized_representative",
    "relationship_other_current_form_maintainer",
    "relationship_other_cost_sharer",
    "relationship_other_history_shape_cost_sharer",
    "relationship_other_history_form_authorized_representative",
    "relationship_other_history_form_maintainer",
    "relationship_partner_total_days_partner",
    "relationship_partner_number_partner_partner_married",
    "relationship_partner_number_partner_partner_unmarried",
    "relationship_partner_current_partner_partner_married",
    "personal_qualities_language",
    "personal_qualities_language_other",
    "personal_qualities_language_requirement_met",
    "personal_qualities_language_requirement_writing_ok",
    "personal_qualities_en_understanding3",
    "personal_qualities_nl_reading3",
    "personal_qualities_nl_reading4",
    "personal_qualities_nl_writing0",
    "personal_qualities_nl_writing1",
    "personal_qualities_nl_writing2",
    "personal_qualities_nl_writing3",
    "personal_qualities_nl_writing_false",
    "personal_qualities_nl_speaking1",
    "personal_qualities_nl_speaking2",
    "personal_qualities_nl_speaking3",
    "address_days_at_address",
    "address_latest_part_rotterdam",
    "address_latest_neighborhood_groot_ijsselmonde",
    "address_latest_neighborhood_new_westen",
    "address_latest_neighborhood_other",
    "address_latest_neighborhood_olde_north",
    "address_latest_neighborhood_vreewijk",
    "address_latest_place_other",
    "address_latest_place_rotterdam",
    "address_latest_district_charlois",
    "address_latest_district_delfshaven",
    "address_latest_district_feijenoord",
    "address_latest_district_ijsselmonde",
    "address_latest_district_kralingen_c",
    "address_latest_district_north",
    "address_latest_district_other",
    "address_latest_district_prins_alexa",
    "address_latest_district_city_center",
    "address_unique_districts_ratio",
    "address_number_different_districts",
    "address_number_personal_records_database_addresses",
    "address_number_mail_address",
    "address_number_residential_address_manual",
    "exemption_days_hist_due to_your_medical_conditions",
    "exemption_reason_hist_medical_grounds",
    "availability_current_deviating_due_to_medical_conditions",
    "availability_number_history_deviating_due to_medical_circumstances",
    "obstacle_days_physical_problems",
    "obstacle_days_psychological_problems",
    "obstacle_hist_physical_problems",
    "obstacle_hist_psychological_problems",
    "personal_qualities_motivation_consultant_judgement",
    "personal_qualities_initiative_consultant_judgement",
    "personal_qualities_presentation_consultant_judgement",
    "personal_qualities_perseverance_consultant_judgement",
    "personal_qualities_flexibility_consultant_judgement",
    "personal_qualities_inquiry_consultant_judgement",
    "personal_qualities_appearance_care_consultant_judgement",
    "personal_qualities_independence_consultant_judgement",
    "personal_qualities_ind_activation_route",
    "personal_qualities_ind_outside_office_hours",
    "personal_qualities_ind_regular_work_rhythm",
]

# --------- LOAD + CLEAN ---------
df = pd.read_csv(DATA_PATH)

# Remove row 1 (contains descriptions)
if 1 in df.index:
    df = df.drop(index=1).reset_index(drop=True)

# Convert label
df[TARGET] = pd.to_numeric(df[TARGET], errors="coerce")
df = df.dropna(subset=[TARGET])
df[TARGET] = df[TARGET].astype(int)

# --- We DO NOT drop any column ---
X = df.drop(columns=[TARGET])
X = X.apply(pd.to_numeric, errors="coerce")
X = X.fillna(0)         # ONNX requires no missing values
y = df[TARGET]

print("Training good model with ALL features intact")

# --------- TRAIN / TEST SPLIT ---------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state= random.randint(1, 100), stratify=y
)

# --------- STEP 1: Compute Reweighting (Fairness) ---------
def compute_fair_weights(X, y, sensitive_cols):
    # merge the sensitive attributes into a single score
    S = X[sensitive_cols].sum(axis=1)
    S_bin = (S > S.median()).astype(int)

    df_tmp = pd.DataFrame({"S": S_bin, "y": y})
    pS = df_tmp["S"].value_counts(normalize=True).to_dict()
    py = df_tmp["y"].value_counts(normalize=True).to_dict()
    pSy = df_tmp.groupby(["S", "y"]).size().div(len(df_tmp)).to_dict()

    weights = []
    for si, yi in zip(df_tmp["S"], df_tmp["y"]):
        numerator = pS[si] * py[yi]
        denom = pSy.get((si, yi), 1e-6)
        weights.append(numerator / denom)

    weights = np.array(weights)
    weights = weights * (len(weights) / weights.sum())  # normalize
    return weights

fair_weights = compute_fair_weights(X_train, y_train, sensitive_features)

# --------- STEP 2: Strong regularization (shrinks proxy effects) ---------
from sklearn.ensemble import HistGradientBoostingClassifier

model = HistGradientBoostingClassifier(
    learning_rate=0.05,
    max_depth=6,
    max_leaf_nodes=32,
    min_samples_leaf=50,
    l2_regularization=0.2,
    random_state=random.randint(1, 100)
)

pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", model)
])

pipeline.fit(X_train, y_train, clf__sample_weight=fair_weights)

# --------- EVALUATION ---------
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred, labels=[0, 1]).ravel()

print("\n=== GOOD MODEL PERFORMANCE ===")
print(f"Accuracy:  {acc:.4f}")
print(f"AUC:       {auc:.4f}")
print(f"TN={tn} FP={fp} FN={fn} TP={tp}")

# --------- EXPORT TO ONNX ---------
initial_type = [("input", FloatTensorType([None, X_train.shape[1]]))]

onnx_model = convert_sklearn(
    pipeline,
    name="good_model_pipeline",
    initial_types=initial_type
)

with open(ONNX_OUTPUT, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"\nSaved GOOD MODEL as: {ONNX_OUTPUT}")


Training good model with ALL features intact

=== GOOD MODEL PERFORMANCE ===
Accuracy:  0.9365
AUC:       0.9659
TN=3392 FP=23 FN=218 TP=161

Saved GOOD MODEL as: model_1.onnx


In [ ]:
from partition_tests import run_partition_tests_for_model